In [167]:
import pandas as pd
import numpy as np


In [168]:
ingredients = pd.read_csv('ingredients.csv')
recipes = pd.read_csv('3_ingredient_recipes.csv')
sources = pd.read_json('sources.json')


In [169]:
ingredients[:5]


,name,source
0,tuna,pond
1,salmon,pond
2,squid,pond
3,seabass,pond
4,sardine,pond


In [170]:
recipes[:5]


,name,type,ingredients
0,tomato and basil soup,starter,"basil, tomato, tomato"
1,checken and leek soup,starter,"chicken, leek, leek"
2,cream of mushroom soup,starter,"bayleaf, milk, mushroom"
3,vegetable chowder,starter,"milk, peas, potato"
4,pea and bacon soup,starter,"pork, peas, peas"


In [171]:
def str2arr(x):
    return str(x).split(', ') if not x is None else x


def arr2count(arr):
    unique, counts = np.unique(arr, return_counts=True)
    return dict(zip(unique, counts))


def lookup_time(x):
    time = 0
    for key, val in x.items():
        sources_df = ingredients[ingredients['name'] == key]['source']
        sources_df = np.array(sources_df)
        if len(sources_df) < 1:
            time += sources['market'].time*val
        else:
            time += sources[sources_df[0]].time*val
    return time


def lookup_cost(x):
    cost = 0
    for key, val in x.items():
        sources_df = ingredients[ingredients['name'] == key]['source']
        sources_df = np.array(sources_df)
        if len(sources_df) < 1:
            cost += sources['market'].cost*val
        else:
            cost += sources[sources_df[0]].cost*val
    return cost


def lookup_difficulty(x):
    difficulty = 0
    for key, val in x.items():
        sources_df = ingredients[ingredients['name'] == key]['source']
        sources_df = np.array(sources_df)
        if len(sources_df) < 1:
            difficulty += sources['market'].difficulty*val
        else:
            difficulty += sources[sources_df[0]].difficulty*val
    return difficulty


In [172]:
recipes['ingredients'] = recipes['ingredients'].apply(lambda x: str2arr(x))
recipes['ingredients'] = recipes['ingredients'].apply(lambda x: arr2count(x))
recipes


,name,type,ingredients
0,tomato and basil soup,starter,"{'basil': 1, 'tomato': 2}"
1,checken and leek soup,starter,"{'chicken': 1, 'leek': 2}"
2,cream of mushroom soup,starter,"{'bayleaf': 1, 'milk': 1, 'mushroom': 1}"
3,vegetable chowder,starter,"{'milk': 1, 'peas': 1, 'potato': 1}"
4,pea and bacon soup,starter,"{'peas': 2, 'pork': 1}"
...,...,...,...
58,watermelon juice,drink,"{'sugar': 1, 'water': 1, 'watermelon': 1}"
59,gods nectar,drink,"{'cherry': 1, 'dragonfruit': 1, 'honey': 1}"
60,coconut cocktail,drink,"{'coconut': 2, 'ice cream': 1}"
61,green tea,drink,"{'tea leaves': 2, 'water': 1}"


In [173]:
print(lookup_time({'basil': 1}))
print(lookup_time({'garlic': 1}))
print(lookup_time({'onion': 1}))
print(lookup_time({'basil': 1, 'garlic': 1, 'onion': 1}))
print(lookup_cost({'basil': 1}))


168
24
24
216
4500


In [174]:
for x in recipes.ingredients[:5]:
    print(x)
    print(lookup_time(x))
    print(lookup_cost(x))
    print(lookup_difficulty(x))
    print('\n')


{'basil': 1, 'tomato': 2}
216
8500
5


{'chicken': 1, 'leek': 2}
504
13500
9


{'bayleaf': 1, 'milk': 1, 'mushroom': 1}
504
13500
9


{'milk': 1, 'peas': 1, 'potato': 1}
360
11000
7


{'peas': 2, 'pork': 1}
504
13500
9




In [175]:
recipes['difficulty'] = recipes.apply(lambda x: lookup_difficulty(x['ingredients']), axis=1)
recipes['time'] = recipes.apply(lambda x: lookup_time(x['ingredients']), axis=1)
recipes['cost'] = recipes.apply(lambda x: lookup_cost(x['ingredients']), axis=1)
recipes = recipes.sort_values(by=['difficulty', 'time', 'cost'], ascending=True)
recipes = recipes.reset_index(drop=True)
recipes


,name,type,ingredients,difficulty,time,cost
0,tuna steak with vegetables,main,"{'carrot': 1, 'potato': 1, 'tuna': 1}",4,96,6000
1,tomato and basil soup,starter,"{'basil': 1, 'tomato': 2}",5,216,8500
2,onion soup,starter,"{'bayleaf': 1, 'garlic': 1, 'onion': 1}",5,216,8500
3,tuna fishcakes,starter,"{'milk': 1, 'potato': 1, 'tuna': 1}",6,240,8500
4,tiger prawn platter,starter,"{'garlic': 1, 'lime': 1, 'prawn': 1}",6,240,8500
...,...,...,...,...,...,...
58,saffron tea,drink,"{'salfron': 1, 'tea leaves': 1, 'water': 1}",9,504,13500
59,gods nectar,drink,"{'cherry': 1, 'dragonfruit': 1, 'honey': 1}",9,504,13500
60,coconut cocktail,drink,"{'coconut': 2, 'ice cream': 1}",9,504,13500
61,green tea,drink,"{'tea leaves': 2, 'water': 1}",9,504,13500


In [176]:
recipes.to_csv('3_optimized_recipes.csv')


In [177]:
top_recipes = recipes.groupby('type').head(3).reset_index(drop=True)
top_recipes['sort_type'] = pd.Categorical(top_recipes['type'], categories=['starter', 'main', 'dessert', 'drink'], ordered=True)
top_recipes = top_recipes.sort_values(by='sort_type', ascending=True)
top_recipes


,name,type,ingredients,difficulty,time,cost,sort_type
1,tomato and basil soup,starter,"{'basil': 1, 'tomato': 2}",5,216,8500,starter
2,onion soup,starter,"{'bayleaf': 1, 'garlic': 1, 'onion': 1}",5,216,8500,starter
3,tuna fishcakes,starter,"{'milk': 1, 'potato': 1, 'tuna': 1}",6,240,8500,starter
0,tuna steak with vegetables,main,"{'carrot': 1, 'potato': 1, 'tuna': 1}",4,96,6000,main
4,margherita pizza,main,"{'cheese': 1, 'flour': 1, 'tomato': 1}",7,360,11000,main
5,roast chicken,main,"{'chicken': 1, 'potato': 1, 'salad': 1}",7,360,11000,main
6,fruit selection,dessert,"{'apple': 1, 'banana': 1, 'strawberry': 1}",7,360,11000,dessert
7,strawberry cheese cake,dessert,"{'cheese': 1, 'flour': 1, 'strawberry': 1}",7,360,11000,dessert
8,pumpkin pie,dessert,"{'flour': 1, 'pumpkin': 1, 'sugar': 1}",7,360,11000,dessert
9,strawberry milkshake,drink,"{'ice cream': 1, 'milk': 1, 'strawberry': 1}",7,360,11000,drink


In [178]:
top_recipes.to_csv('3_optimized_top_recipes.csv')
